# Mount drive and install packages

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/dataset/

/content/drive/MyDrive/dataset


In [4]:
import time
import requests
!pip install simplejson
import simplejson as json
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.7 MB/s eta 0:00:00


# Cleaning

In [5]:
input = "addresses.csv"
headerOfCol = "Dropoff Address"
output = "latlongcoordinates.csv"

In [6]:
data = pd.read_csv(input, dtype=str, encoding = 'latin-1')

In [7]:
# Extracts the postal code from the address

data['Postal Code'] = data[headerOfCol].str.extract(r'(\d{6})')
data['Postal Code'] = pd.to_numeric(data['Postal Code'])

In [8]:
# Output for cleaning

data.to_csv(output, index = False)

### Download the file from Google Drive, fill in the missing postal codes/remove the rows and reupload the file to Google Drive

# Geocoordinates

In [9]:
input = output
new_output = "geocoordinates.csv"

In [10]:
# Read in data

df_postcode=pd.read_csv(input, dtype=str, encoding = 'latin-1')
df_postcode['Postal Code']=df_postcode['Postal Code'].astype(str)

In [11]:
# Drop first column

df_postcode = df_postcode.drop(df_postcode.columns[0], axis=1)
df_postcode = df_postcode.groupby('Postal Code').size().reset_index(name='Frequency')

In [ ]:
# For loop to send postal codes to OneMAP API
# Input: Postal Codes as string format
# Output: Longitude and Latitude coordinates

for i in range(len(df_postcode)):
    if len(str(df_postcode['Postal Code'][i]))==5:
        df_postcode['Postal Code'][i]='0'+df_postcode['Postal Code'][i]
df_postcode['OnemapLongitude'] = pd.Series(0, index=df_postcode.index)
df_postcode['OnemapLatitude'] = pd.Series(0, index=df_postcode.index)

start=time.time()
for i in range(len(df_postcode)):
    req=requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+df_postcode['Postal Code'][i]+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    jdata = json.loads(req.text)
    if jdata['found']>=1:
        df_postcode['OnemapLongitude'][i]=jdata['results'][0]['LONGITUDE']
        df_postcode['OnemapLatitude'][i]=jdata['results'][0]['LATITUDE']
    else:
        df_postcode['OnemapLongitude'][i]=float('nan')
        df_postcode['OnemapLatitude'][i]=float('nan')
df_postcode.to_csv(new_output, index = False)
print('Time Taken:',time.time()-start)

<ipython-input-12-78e267957efa>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_postcode['OnemapLongitude'][i]=float('nan')
<ipython-input-12-78e267957efa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_postcode['OnemapLatitude'][i]=float('nan')
